In [5]:
import sounddevice as sd
print(sd.query_devices())

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Microphone Array (Realtek(R) Au, MME (2 in, 0 out)
   2 Headset (realme Buds T310), MME (1 in, 0 out)
   3 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  4 Headphones (realme Buds T310), MME (0 in, 2 out)
   5 Speaker (Realtek(R) Audio), MME (0 in, 2 out)
   6 Primary Sound Capture Driver, Windows DirectSound (2 in, 0 out)
   7 Microphone Array (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
   8 Headset (realme Buds T310), Windows DirectSound (1 in, 0 out)
   9 Primary Sound Driver, Windows DirectSound (0 in, 2 out)
  10 Headphones (realme Buds T310), Windows DirectSound (0 in, 2 out)
  11 Speaker (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
  12 Headphones (realme Buds T310), Windows WASAPI (0 in, 2 out)
  13 Speaker (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
  14 Microphone Array (Realtek(R) Audio), Windows WASAPI (2 in, 0 out)
  15 Headset (realme Buds T310), Windows WASAPI (1 in, 0 out)
  16 He

**using vosk converting live audio into text**

In [ ]:
import sounddevice as sd
import queue
import json
import time
import wave
from vosk import Model, KaldiRecognizer

model_path = 'D:/Infosys_Springboard/AI_Internship/vosk-model'
vosk_model = Model(model_path)
rec = KaldiRecognizer(vosk_model, 16000)

q = queue.Queue()

duration = 10  # seconds

# File where audio will be saved
output_file = "recorded_audio.wav"

wf = wave.open(output_file,'wb')
wf.setnchannels(1)
wf.setsampwidth(2)
wf.setframerate(16000)

def callback(indata, frames, time_info, status):
    q.put(bytes(indata))

with sd.RawInputStream(
    samplerate=16000,
    blocksize=4000,
    dtype='int16',
    channels=1,
    callback=callback
):
    print("Listening... speak now")

    start = time.time()
    result = ""

    while time.time() - start < duration:
        data = q.get()

        wf.writeframes(data)

        if rec.AcceptWaveform(data):
            text = json.loads(rec.Result()).get("text")
            if text:
                result += text + " "

    # Final cleanup recognition
    final_text = json.loads(rec.FinalResult()).get("text")
    if final_text:
        result += final_text

wf.close()

print("\nFinal Output:\n", result)


Listening... speak now

Final Output:
 mermaid the is turns out the real nice be will give him 


**using whisper model to convert live audio into text**

In [ ]:
import sounddevice as sd
import numpy as np
from faster_whisper import WhisperModel
import time
import torch
import queue

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = WhisperModel("small", device=device)

samplerate = 16000
duration = 10
q = queue.Queue()

def callback(indata, frames, time_info, status):
    q.put(indata.copy())

audio_buffer = []

with sd.InputStream(
    samplerate=samplerate,
    blocksize=4000,
    channels=1,
    dtype="float32",
    callback=callback
):
    print("Listening...")
    start = time.time()

    while time.time() - start < duration:
        audio_buffer.append(q.get())

print("Stopped recording.")

# Combine chunks
audio_data = np.concatenate(audio_buffer, axis=0).flatten()

# Transcribe whole audio
segments, _ = model.transcribe(audio_data)

result_text = " ".join([s.text for s in segments])
print("Transcription:\n", result_text)


d:\Infosys_Springboard\AI_Internship\venv\lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
d:\Infosys_Springboard\AI_Internship\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
